In [12]:
# Import geometry features, NGSolve and xfem
from netgen.geom2d import SplineGeometry
from ngsolve import *
from xfem import *
from math import pi
# Visualisation
from ngsolve.webgui import *
DrawDC = MakeDiscontinuousDraw(Draw)

interactive = 1

In [2]:
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
h0 = 1/10

In [3]:
# Construct background mesh
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((-1, -0.5), (1, 0.5), bc=1)
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# mesh = MakeStructured2DMesh(quads=False, nx=8, ny=8)
# Draw(mesh)

In [7]:
circle1 = 1/9-(x-0.5)**2-y**2
circle2 = 1/9-(x+0.5)**2-y**2
level_sets = (circle1, circle2)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(lsets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
    DrawDC(lset_p1,-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [11]:
# multi-levelsets 后的区域可视化
Draw(CoefficientFunction((lsets_p1[0],lsets_p1[1],0,0)), mesh, 'NEG-NEG', 
     min=-1, max=1, 
     eval_function="value.x < 0.0? (value.y < 0.0? 1.0 : -1.0) : -1.0")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [17]:
# 面积分
area = Integrate(CoefficientFunction(1) * dCut(lsets_p1, (NEG, NEG), order=0), mesh=mesh)
error = abs(area - (2-2/9*pi))
print("Result of the integration: {}".format(area))
print("Error of the integration: {:5.3e}".format(error))

Result of the integration: 1.317610919393547
Error of the integration: 1.574e-02


In [18]:
# 线积分
len_c1 = Integrate(CoefficientFunction(1)*dCut(lsets_p1,(IF,NEG),order=0),mesh)
error_c1 = abs(len_c1 - 2/3*pi)
print("Result of the integration: {}".format(len_c1))
print("Error of the integration: {:5.3e}".format(error_c1))

Result of the integration: 2.0761171268678327
Error of the integration: 1.828e-02


In [16]:
# print(help(dCut))

# 用 Convenience layer 管理多个水平集组成的复杂几何


In [20]:
from xfem.mlset import *

# DomainTypeArray?

In [25]:
# 使用区域类型组合初始化 DomainTypeArray
# 比如选取第一个levelset的任意区域和第二个levelset的负区域
domain = DomainTypeArray((ANY,NEG)) 

# 但是注意这里 ANY 不是一个具体的区域，是一个 COMBINED_DOMAIN_TYPES， 所以不能直接用于积分
# 而是要展开成每个单独的区域再去积分
# 因此可以查看这里domain的所有区域list
# 结果表示第一个levelset的任意区域其实就是既包括正的部分，也包括负的部分
print(domain.as_list)

# 也可以查看区域的余维数
print(domain.codim)

[(<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.NEG: 0>), (<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.NEG: 0>)]
0


## DomainTypeArray 可以当成“几何集合”来做布尔运算
- DomainTypeArray = 多个 level set 的符号组合（NEG/POS/IF）构成的几何区域
- 可以对它做 NOT、UNION、INTERSECTION：
    - ~ （NOT）：取补集
    - \| （UNION）：取并集
    - &（INTERSECTION）：取交集 （注意取交集可能会增加codimension）

In [27]:
circles = DomainTypeArray((NEG,NEG))
computation_domain = ~circles
computation_domain.as_list

[(<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.NEG: 0>),
 (<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.POS: 1>),
 (<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.POS: 1>)]

## 生成边界

In [30]:
boundary = circles.Boundary()
boundary.as_list

[(<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.IF: 2>),
 (<DOMAIN_TYPE.IF: 2>, <DOMAIN_TYPE.NEG: 0>)]

## 可视化
DomainTypeArray 描述的是“区域”，可以使用 .Indicator(lsets)处理codim=0的区域 和 .IndicatorSmoothed(lsets,eps)处理codim>0的区域。

这些函数返回一个CoefficientFunction满足：
- 区域内部 = 1
- 区域外部 = 0
- 对于 codim > 0（面、线、点），.IndicatorSmoothed() 会显示一个值为1的“带状区域”。
这两个函数主要用于 调试（debugging） 和 可视化（visualization）。

In [32]:
DrawDC(circles.Indicator(lsets_p1),-1,1,mesh,'circles_indicator')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [33]:
DrawDC(boundary.IndicatorSmoothed(lsets_p1, 0.01), -3.5, 2.5, mesh,
       "boundary_indicator")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## 去除不必要的区域

In [ ]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
neg = ci.GetElementsOfType(NEG)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(neg),mesh)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [ ]:
print(type(ba_facets))

In [ ]:
vtk = VTKOutput(mesh,[levelset,BitArrayCF(hasneg),BitArrayCF(hasif)],['levelset','active_mesh','boundary'],"/mnt/d/ngs_output/Biot_Brinkman/heart_domain",subdivision=0)
# vtk = VTKOutput(mesh,[levelset_domain],['levelset_domain'],"/mnt/d/ngs_output/Biot_Brinkman/levelset_domain",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(neg)],['uncutElements'],"/mnt/d/ngs_output/Biot_Brinkman/uncutElements",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(ba_facets)],['ghost_faces'],"/mnt/d/ngs_output/Biot_Brinkman/ghost_faces",subdivision=0)
vtk.Do()   #输出网格

In [ ]:
print(help(Draw))